In [1]:
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_687194/327059659.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%autoreload

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import scripts
print(scripts.__path__)
import pathlib


from scripts.common.schemas import TypeCollectionCategory
from scripts.infer.structure import DatasetFolderStructure

dataset = DatasetFolderStructure(pathlib.Path(
    "/nfs/home/bsparks/mdti4py/datasets/better-types-4-py-dataset"
))
assert dataset.dataset_root.is_dir(), f"{dataset.dataset_root} not a directory!"

print(dataset)

['/nfs/home/bsparks/mdti4py/scripts/scripts']


In [ ]:
import importlib
import experiments.inferred

In [ ]:
artifact_root = pathlib.Path("/nfs/home/bsparks/mdti4py/datasets")
assert artifact_root.is_dir(), f"Cannot find {artifact_root=}"

# Ground Truth

In [ ]:
importlib.reload(experiments.inferred)
groundtruth = experiments.inferred.load_groundtruths(artifact_root, dataset)

In [ ]:
groundtruth[groundtruth["file"] == "aiosonic/pools.py"].drop(columns=["category", "qname", "repository"])

In [ ]:
trivial_mask = groundtruth["adjusted_anno"].isin(["None", "Any"])
groundtruth = groundtruth[~trivial_mask]

groundtruth

In [ ]:
print("=== Label Counts === ")
print(f"{groundtruth.raw_anno.notna().sum()=}, {groundtruth.adjusted_anno.notna().sum()=}, {groundtruth.base_anno.notna().sum()=}")

In [ ]:
from scripts.common.schemas import ExtendedTypeCollectionSchema

import experiments.inferred
from experiments.predictions import ubiquitous_types, co_occurrences

import pandera.typing as pt, pandas as pd, seaborn as sns
import importlib

import matplotlib.pyplot as plt

def adjusted_pipeline(tool: str, groundtruth: pt.DataFrame[ExtendedTypeCollectionSchema]) -> pd.DataFrame:
    importlib.reload(experiments.inferred)
    print(f"Loaded inferred")
    
    inferred = experiments.inferred.load_entire_inferred(artifact_root, dataset, tool_name=tool, task="all")
    experiments.inferred.error_if_duplicate_keys(inferred)
    print(f"No duplicate keys found")

    adjusted = experiments.inferred.typet5_adjusted_form(inferred)
    print(f"Converted to adjusted form")

    trivial_mask = groundtruth.adjusted_anno.str.endswith((".None", ".Any")) | groundtruth.adjusted_anno.isin(["None", "Any"])
    groundtruth = groundtruth[~trivial_mask]
    print("Removed trivial types from groundtruth (None and Any)")

    aligned = experiments.inferred.join_truth_to_preds(
        truth=groundtruth, 
        predictions=adjusted, 
        comparable_anno=ExtendedTypeCollectionSchema.adjusted_anno,
    )
    print(f"{aligned.shape}")
    print(f"Joined ground truth to predictions")
    
    evaluatable = experiments.inferred.evaluatable(aligned)
    assert evaluatable["gt_anno"].notna().all()
    assert evaluatable["anno"].notna().all()
    
    print(f"Reduced to evaluatable: {evaluatable.shape}")
    return evaluatable

def base_pipeline(tool: str, groundtruth: pt.DataFrame[ExtendedTypeCollectionSchema]) -> pd.DataFrame:
    importlib.reload(experiments.inferred)   
    inferred = experiments.inferred.load_entire_inferred(artifact_root, dataset, tool_name=tool, task="all")
    print(f"Loaded inferred")
    
    experiments.inferred.error_if_duplicate_keys(inferred)
    print(f"No duplicate keys found")

    adjusted = experiments.inferred.typet5_base_form(inferred)
    print(f"Converted to base form")

    trivial_mask = groundtruth.base_anno.isin(["None", "Any"])
    groundtruth = groundtruth[~trivial_mask]
    print("Removed trivial types from groundtruth (None and Any)")

    aligned = experiments.inferred.join_truth_to_preds(
        truth=groundtruth, 
        predictions=adjusted, 
        comparable_anno=ExtendedTypeCollectionSchema.base_anno,
    )
    print(f"{aligned.shape}")
    print(f"Joined ground truth to predictions")
    
    evaluatable = experiments.inferred.evaluatable(aligned)
    assert evaluatable["gt_anno"].notna().all()
    assert evaluatable["anno"].notna().all()
    
    print(f"Reduced to evaluatable: {evaluatable.shape}")
    return evaluatable

# HiTyperNoML

In [ ]:
hitypernoml_adjusted = adjusted_pipeline(tool="HiTyperNoML", groundtruth=groundtruth)

In [ ]:
hitypernoml_base = base_pipeline(tool="HiTyperNoML", groundtruth=groundtruth)

In [ ]:
experiments.predictions.performance(hitypernoml_adjusted.replace("<MISSING>", pd.NA), total=True)

In [ ]:
experiments.predictions.performance(hitypernoml_base.replace("<MISSING>", pd.NA), total=True)

In [ ]:
thres_figs, ax1 = plt.subplots(nrows=1)

importlib.reload(experiments.predictions)
hitypernoml_base_ubiq = experiments.predictions.ubiquitous_types(hitypernoml_base.replace("<MISSING>", pd.NA))

ax1.set_title('HiTyperNoML')
co_occurrences(hitypernoml_base_ubiq, truth="gt_anno", pred="anno", threshold=0.08, figsize=(5, 10), ax=ax1)

plt.tight_layout()
plt.show()